In [66]:
import pandas as pd

In [67]:
file_id = "1lfH64MX8NHuxn7745leZ6LaXRVLAAer77J336ZFOTIk"
gid = "1900938527"  # onglet cible
url = f"https://docs.google.com/spreadsheets/d/{file_id}/export?format=csv&gid={gid}"

df = pd.read_csv(url)  # éventuellement: encoding="utf-8"
df_clean = pd.DataFrame()
print(df.shape)


(46, 30)


In [68]:
df_clean = df

In [69]:
df_clean.columns

Index(['Horodateur', 'Quel est votre nationalité?',
       '  Dans quel pays résidez-vous actuellement ?  ',
       'Quelle est votre tranche d’âge ?  ',
       'Quelle est votre situation familiale ? ',
       'Quelle est votre tranche de revenus mensuels nets du foyer ? ',
       'À quelle fréquence voyagez vous à l’étranger (hors Europe) ?  ',
       'Avez-vous déjà voyagé au Japon ?  ',
       'Quelle durée de séjour avez-vous prévue ?  ',
       'Quelles régions du Japon vous intéressent le plus ? (Choisissez 3 max.)  ',
       'À quel point ces motivations influencent elles votre envie de voyager au Japon ? [Découverte de la culture et de l’histoire (temples, traditions, samouraïs, geishas, etc.)]',
       'À quel point ces motivations influencent elles votre envie de voyager au Japon ? [Gastronomie japonaise (sushis, ramen, wagyu, street food, etc.)]',
       'À quel point ces motivations influencent elles votre envie de voyager au Japon ? [Paysages naturels et randonnées (monta

In [70]:
df_clean = df_clean.rename(columns={
    "Quel est votre nationalité?": "nationality",
    "  Dans quel pays résidez-vous actuellement ?  ": "country",
    "Quelle est votre tranche d’âge ?  ": "age_group",
    "Quelle est votre situation familiale ? ": "family_situation",
    "Quelle est votre tranche de revenus mensuels nets du foyer ? ": "household_income_in_€",
    "À quelle fréquence voyagez vous à l’étranger (hors Europe) ?  ": "travel_frequency",
    "Avez-vous déjà voyagé au Japon ?  ": "been_to_Japan",
    "Quelle durée de séjour avez-vous prévue ?  ": "Japan_vac_duration",
    "Quelles régions du Japon vous intéressent le plus ? (Choisissez 3 max.)  ": "most_wanted_pref_to_visit",
    "À quel point ces motivations influencent elles votre envie de voyager au Japon ? [Découverte de la culture et de l’histoire (temples, traditions, samouraïs, geishas, etc.)]": "rating_interest_culture_and_history",
    "À quel point ces motivations influencent elles votre envie de voyager au Japon ? [Gastronomie japonaise (sushis, ramen, wagyu, street food, etc.)]": "rating_interest_food",
    "À quel point ces motivations influencent elles votre envie de voyager au Japon ? [Paysages naturels et randonnées (montagnes, volcans, cerisiers en fleurs, etc.)]": "rating_interest_nature_hiking",
    "À quel point ces motivations influencent elles votre envie de voyager au Japon ? [Technologie, innovation et shopping (Tokyo high-tech, Akihabara, mode, etc.)]": "rating_interest_shopping_and_techno",
    "À quel point ces motivations influencent elles votre envie de voyager au Japon ? [Festivals et événements (matsuri, concerts, sport, sumo, etc.)]": "rating_interest_events_and_festivals",
    "À quel point ces motivations influencent elles votre envie de voyager au Japon ? [Bien-être (onsen, ryokan, détente)]": "rating_interest_welness",
    "À quel point ces motivations influencent elles votre envie de voyager au Japon ? [Parc d'attraction (Disneyland, Universal...)]": "rating_interest_theme_park",
    "Quel budget global prévoyez vous pour un voyage au Japon (par personne et par semaine , hors vol international ) ?  ": "budget_per_week",
    "Parmi les types d’hébergement suivants, lequel correspond le mieux à vos préférences principales pour un séjour au Japon ? ": "Japan_prefered_accomodation",
    "Quels sont les principaux freins ou difficultés que vous rencontrez (ou pourriez rencontrer) lors d’un voyage au Japon ? (Choisissez 3 max.)  ": "Japan_most_difficulties",
    "Si vous ne pouviez pas voyager au Japon, quelle destination alternative choisiriez-vous ?": "alternative_destination",
    "Quelle a été la principale raison pour laquelle vous auriez choisi cette destination plutôt que le Japon ? ": "alt_dest_main_reason",
    "Parmi les types d’hébergement suivants, lequel correspond le mieux à vos préférences principales lors de vos voyages dans d’autres pays (hors Japon)   ? ": "alt_dest_prefered_accomodation",
    "Lors de vos voyages dans d’autres pays (hors Japon), quel est votre budget moyen par semaine et par personne , hors vol international ?  ": "alt_dest_budget_per_week",
    "Lors de vos voyages dans d’autres pays (hors Japon), quel(s) mode(s) de transport utilisez-vous le plus souvent ?": "alt_dest_transportation",
    "Comment préparez-vous vos voyages en général ? (Multiple choix possible)": "trip_prep",
    "Quel canal utilisez-vous le plus pour réserver vos voyages ?  ": "booking_trip_channel",
    "Parmi les éléments suivants, lequel influence le plus votre choix de destination de vacances (hors Japon) ? ": "most_influencial_reason_to_choose_dest",
    "Lorsque vous voyagez en dehors du Japon, quelles sont les principales difficultés que vous rencontrez habituellement ?\n(Choisissez jusqu’à 3 réponses)": "alt_dest_most_difficulties",
    "Qu’est-ce qui rendrait le Japon plus attractif comme destination pour vous ?  ": "recomendation_to_improve_attractiveness"
})

In [71]:
def normalize_text(s):
    if pd.isna(s): return s
    return (str(s)
            .strip()
            .lower()
            .replace("é","e").replace("è", "e").replace("ê", "e")
            .replace("à","a").replace("ï", "i").replace("î", "i")
            .replace("$","").replace("€", ""))

mapping = {
    #France
    "france": "France",
    "française": "France",
    "français": "France",
    "francais": "France",
    "francaise": "France",
    "french": "France",

    #China
    "chine": "China",
    "chinoise": "China",
    "chinois": "China",
    "中国": "China",
    "法国": "China",
    "chinese": "China",

    #Taiwan
    "taïwan ": "Taiwan",
    "taiwan": "Taiwan",
    "taiwanais": "Taiwan",
    "taiwanaise": "Taiwan",

    #Vietnam
    "vietnam": "Vietnam",
    "vietnamien": "Vietnam",
    "vietnamienne": "Vietnam",

    #Portugal
    "portuguese": "Portugal",

    #Israel
    "israel": "Israel",
    "israël": "Israel",
    "israelien": "Israel",
    "israelienne": "Israel",
    "israélien": "Israel",

    #Spain
    "espagne": "Spain",

    #Germany
    "allemagne": "Germany",

}

if "nationality" in df_clean.columns:
    df_clean["nationality"] = (df_clean["nationality"]
                               .map(normalize_text)
                               .map(mapping)
                               .fillna(df_clean["nationality"]))
    
if "country" in df_clean.columns:
    df_clean["country"] = (df_clean["country"]
                               .map(normalize_text)
                               .map(mapping)
                               .fillna(df_clean["country"]))

In [72]:
df_clean.columns

Index(['Horodateur', 'nationality', 'country', 'age_group', 'family_situation',
       'household_income_in_€', 'travel_frequency', 'been_to_Japan',
       'Japan_vac_duration', 'most_wanted_pref_to_visit',
       'rating_interest_culture_and_history', 'rating_interest_food',
       'rating_interest_nature_hiking', 'rating_interest_shopping_and_techno',
       'rating_interest_events_and_festivals', 'rating_interest_welness',
       'rating_interest_theme_park', 'budget_per_week',
       'Japan_prefered_accomodation', 'Japan_most_difficulties',
       'alternative_destination', 'alt_dest_main_reason',
       'alt_dest_prefered_accomodation', 'alt_dest_budget_per_week',
       'alt_dest_transportation', 'trip_prep', 'booking_trip_channel',
       'most_influencial_reason_to_choose_dest', 'alt_dest_most_difficulties',
       'recomendation_to_improve_attractiveness'],
      dtype='object')

In [73]:
print(df_clean["nationality"].value_counts())
print(df_clean["country"].value_counts())

nationality
France      32
China       10
Vietnam      1
Taiwan       1
Israel       1
Portugal     1
Name: count, dtype: int64
country
France      41
Spain        2
Germany      1
China        1
Portugal     1
Name: count, dtype: int64


In [74]:
print(df_clean["age_group"].value_counts())

age_group
25 – 34 ans        16
35 – 44 ans        13
45 – 54 ans         9
18 – 24 ans         3
55 – 64 ans         2
35–44 years old     2
18–24 years old     1
Name: count, dtype: int64


In [75]:
def clean_age (age):
    if pd.isna(age):
        return none
    age = str(age).strip()
    if age.startswith("18"):
        return "18-24"
    elif age.startswith("25"):
        return "25-34"
    elif age.startswith("35"):
        return "35-44"
    elif age.startswith("45"):
        return "45-54"
    elif age.startswith("55"):
        return "55-64"
    elif age.startswith("65"):
        return "65 and over"
    else:
        return "18 and less"

df_clean["age_group"] = df_clean["age_group"].apply(clean_age)


In [76]:
print(df_clean["age_group"].value_counts())

age_group
25-34    16
35-44    15
45-54     9
18-24     4
55-64     2
Name: count, dtype: int64


In [77]:
print(df_clean["family_situation"].value_counts())

family_situation
Célibataire                                        15
Marié(e)/Pacsé(e) avec enfant(s)                    9
En couple avec enfant(s)                            7
Marié(e)/Pacsé(e) sans enfant                       6
En couple sans enfant                               5
Préfère ne pas répondre                             1
Married / in a civil partnership, no children       1
Married / in a civil partnership, with children     1
In a relationship, no children                      1
Name: count, dtype: int64


In [78]:
family_situation_map = {
    
    "single": "Single",
    "celibataire": "Single",

    "en couple sans enfant": "Relationship_no_kids",
    "in a relationship, no children": "Relationship_no_kids",

    "en couple avec enfant(s)": "Relationship_with_kids",
    "in a relationship, with children": "Relationship_with_kids",

    "marie(e)/pacse(e) sans enfant": "Married_no_kids",
    "married / in a civil partnership, no children": "Married_no_kids",

    "marie(e)/pacse(e) avec enfant(s)": "Married_with_kids",
    "married / in a civil partnership, with children": "Married_with_kids",

    "prefere ne pas repondre": "Unknown",
    "prefer not to say": "Unknown"
}

df_clean["family_situation"] = (df_clean["family_situation"]
                            .map(normalize_text)
                            .map(family_situation_map)
                            .fillna(df_clean["family_situation"]))


In [79]:
print(df_clean["family_situation"].value_counts())

family_situation
Single                    15
Married_with_kids         10
Married_no_kids            7
Relationship_with_kids     7
Relationship_no_kids       6
Unknown                    1
Name: count, dtype: int64


In [80]:
print(df_clean["household_income_in_€"].value_counts())

household_income_in_€
1 500 – 1 999 €                       10
3 000 – 3 999 €                        6
2 000 – 2 499 €                        5
2 500 – 2 999 €                        5
5000 – 5 999 €                         5
Supérieur à 7 000 €                    3
Préfère ne pas répondre                3
Moins de 1 500 €                       3
4 000 – 4 999 €                        2
6 000 – 6 999 €                        1
$1,700 – $2,200 (~€1,500 – €1,999)     1
$3,400 – $4,400 (~€3,000 – €3,999)     1
Prefer not to say                      1
Name: count, dtype: int64


In [81]:
clean_income = {
    
    "moins de 1 500 ": "1500 and less",
    "less than 1,700 (~1,500)": "1500 and less",

    "1 500 – 1 999 ": "1500-1999",
    "1,700 – 2,200 (~1,500 – 1,999)": "1500-1999",

    "2 000 – 2 499 ": "2000-2499",
    "2,200 – 2,700 (~2,000 – 2,499)": "2000-2499",

    "2 500 – 2 999 ": "2500-2999",
    "2,800 – 3,300 (~2,500 – 2,999)": "2500-2999",

    "3 000 – 3 999 ": "3000-3999",
    "3,400 – 4,400 (~3,000 – 3,999)": "3000-3999",

    "4 000 – 4 999 ": "4000–4999",
    "4,500 – 5,500 (~4,000 – 4,999)": "4000–4999",

    "5000 – 5 999 ": "5000–5999",
    "5,600 – 6,600 (~5,000 – 5,999)": "5000–5999",

    "6 000 – 6 999 ": "6000–6999",
    "6,700 – 7,700 (~6,000 – 6,999)": "6000–6999",

    "superieur a 7 000 ": "7000 and more",
    "more than 7,800 (~7,000+)": "7000 and more",

    "prefere ne pas repondre": "Unknown",
    "prefer not to say": "Unknown",
    }

df_clean["household_income_in_€"] = (df_clean["household_income_in_€"]
                            .map(normalize_text)
                            .map(clean_income)
                            .fillna(df_clean["household_income_in_€"]))


In [82]:
print(df_clean["household_income_in_€"].value_counts())

household_income_in_€
1500-1999        11
3000-3999         7
2000-2499         5
2500-2999         5
5000–5999         5
Unknown           4
7000 and more     3
1500 and less     3
4000–4999         2
6000–6999         1
Name: count, dtype: int64


In [83]:
print(df_clean["travel_frequency"].value_counts())

travel_frequency
Tous les 2–3 ans                   14
Une fois par an                    14
Plusieurs fois par an               6
Une fois tous les 5 ans ou plus     5
Jamais                              4
Every 2–3 years                     2
Once every 5 years or more          1
Name: count, dtype: int64


In [84]:
clean_travel_frequency = {
    
    "jamais": "Never",
    "une fois tous les 5 ans ou plus": "Once every 5 years or more",
    "tous les 2–3 ans": "Every 2–3 years",
    "une fois par an": "Once a year",
    "plusieurs fois par an": "Several times a year",
    }

df_clean["travel_frequency"] = (df_clean["travel_frequency"]
                            .map(normalize_text)
                            .map(clean_travel_frequency)
                            .fillna(df_clean["travel_frequency"]))


In [85]:
print(df_clean["travel_frequency"].value_counts())

travel_frequency
Every 2–3 years               16
Once a year                   14
Several times a year           6
Once every 5 years or more     6
Never                          4
Name: count, dtype: int64


In [86]:
print(df_clean["been_to_Japan"].value_counts())

been_to_Japan
Non, mais j’aimerais y aller        29
Non, et je ne suis pas intéressé     7
Oui, une fois                        5
Oui, plusieurs fois                  2
No, but I would like to go           2
Yes, several times                   1
Name: count, dtype: int64


In [87]:
clean_been_to_japan = {

    "oui, une fois": "Yes, once",
    "oui, plusieurs fois": "Yes, several times",
    "non, mais j’aimerais y aller": "No, but I would like to go",
    "non, et je ne suis pas interesse": "No, and I’m not interested",
    }

df_clean["been_to_Japan"] = (df_clean["been_to_Japan"]
                            .map(normalize_text)
                            .map(clean_been_to_japan)
                            .fillna(df_clean["been_to_Japan"]))

print(df_clean["been_to_Japan"].value_counts())

been_to_Japan
No, but I would like to go    31
No, and I’m not interested     7
Yes, once                      5
Yes, several times             3
Name: count, dtype: int64


In [88]:
print(df_clean["Japan_vac_duration"].value_counts())

Japan_vac_duration
2 semaines                              15
3 semaines                               8
1 semaine                                5
Je ne sais pas / Pas assez renseigné     5
4 semaines                               2
2 weeks                                  2
Plus de 4 semaines                       1
More than 4 weeks                        1
Name: count, dtype: int64


In [89]:
clean_Japan_vac_duration = {
    
    "1 semaine": "1 week",
    "2 semaines": "2 weeks",
    "3 semaines": "3 weeks",
    "4 semaines": "4 weeks",
    "plus de 4 semaines": "More than 4 weeks",
    "je ne sais pas / pas assez renseigne": "I don’t know yet / Not sure"
    }

df_clean["Japan_vac_duration"] = (df_clean["Japan_vac_duration"]
                            .map(normalize_text)
                            .map(clean_Japan_vac_duration)
                            .fillna(df_clean["Japan_vac_duration"]))

print(df_clean["Japan_vac_duration"].value_counts())

Japan_vac_duration
2 weeks                        17
3 weeks                         8
1 week                          5
I don’t know yet / Not sure     5
More than 4 weeks               2
4 weeks                         2
Name: count, dtype: int64


In [90]:
print(df_clean["most_wanted_pref_to_visit"].value_counts())

most_wanted_pref_to_visit
Je n’ai pas encore d’idée précise, j’ai besoin d’y réfléchir ou de me renseigner.                                                                                                                                                                                                                                                    11
Tokyo et sa région (Kanto), Kyoto / Osaka / Nara (Kansai)                                                                                                                                                                                                                                                                             5
Tokyo et sa région (Kanto), Kyoto / Osaka / Nara (Kansai), Chūbu (Nagoya, Alpes japonaises, Kanazawa, Takayama, Mont Fuji côté Yamanashi/Shizuoka)                                                                                                                                                                            

In [95]:
clean_most_wanted_pref_to_visit = {
    
    "tokyo et sa region (kanto)": "Kanto",
    "tokyo and its region (kanto)": "Kanto",

    "kyoto / osaka / nara (kansai)": "Kansai",

    "region du tohoku (ex. yamagata, sendai)": "Tohoku",
    "tohoku region (e.g. yamagata, sendai)": "Tohoku",

    "chūgoku (hiroshima, miyajima, okayama, matsue)": "Chūgoku",

    "shikoku (île du pelerinage des 88 temples, matsuyama, iya valley)": "Shikoku",
    "shikoku (88 temple pilgrimage island, matsuyama, iya valley)": "Shikoku",

    "chūbu (nagoya, alpes japonaises, kanazawa, takayama, mont fuji cote yamanashi/shizuoka)": "Chūbu",
    "chubu (nagoya, japanese alps, kanazawa, takayama, mt. fuji – yamanashi/shizuoka side)": "Chūbu",

    "je n’ai pas encore d’idee precise, j’ai besoin d’y reflechir ou de me renseigner.": "Unknown",
    "i don’t have a clear idea yet / i need to think or find out more": "Unknown",
    }

df_clean["most_wanted_pref_to_visit"] = (df_clean["most_wanted_pref_to_visit"]
                            .map(normalize_text)
                            .map(clean_most_wanted_pref_to_visit)
                            .fillna(df_clean["most_wanted_pref_to_visit"]))

print(df_clean["most_wanted_pref_to_visit"].value_counts())

most_wanted_pref_to_visit
Unknown                                                                                                                                                                                                                                                                                                                              12
Tokyo et sa région (Kanto), Kyoto / Osaka / Nara (Kansai)                                                                                                                                                                                                                                                                             5
Tokyo et sa région (Kanto), Kyoto / Osaka / Nara (Kansai), Chūbu (Nagoya, Alpes japonaises, Kanazawa, Takayama, Mont Fuji côté Yamanashi/Shizuoka)                                                                                                                                                                            

In [92]:
print(df_clean["household_income_in_€"].value_counts())

clean_income = {
    
    "moins de 1 500 ": "1500 and less",
    "less than 1,700 (~1,500)": "1500 and less",

    "prefere ne pas repondre": "Unknown",
    "prefer not to say": "Unknown",
    }

df_clean["household_income_in_€"] = (df_clean["household_income_in_€"]
                            .map(normalize_text)
                            .map(clean_income)
                            .fillna(df_clean["household_income_in_€"]))

print(df_clean["household_income_in_€"].value_counts())

household_income_in_€
1500-1999        11
3000-3999         7
2000-2499         5
2500-2999         5
5000–5999         5
Unknown           4
7000 and more     3
1500 and less     3
4000–4999         2
6000–6999         1
Name: count, dtype: int64
household_income_in_€
1500-1999        11
3000-3999         7
2000-2499         5
2500-2999         5
5000–5999         5
Unknown           4
7000 and more     3
1500 and less     3
4000–4999         2
6000–6999         1
Name: count, dtype: int64


In [93]:
df_clean.columns

Index(['Horodateur', 'nationality', 'country', 'age_group', 'family_situation',
       'household_income_in_€', 'travel_frequency', 'been_to_Japan',
       'Japan_vac_duration', 'most_wanted_pref_to_visit',
       'rating_interest_culture_and_history', 'rating_interest_food',
       'rating_interest_nature_hiking', 'rating_interest_shopping_and_techno',
       'rating_interest_events_and_festivals', 'rating_interest_welness',
       'rating_interest_theme_park', 'budget_per_week',
       'Japan_prefered_accomodation', 'Japan_most_difficulties',
       'alternative_destination', 'alt_dest_main_reason',
       'alt_dest_prefered_accomodation', 'alt_dest_budget_per_week',
       'alt_dest_transportation', 'trip_prep', 'booking_trip_channel',
       'most_influencial_reason_to_choose_dest', 'alt_dest_most_difficulties',
       'recomendation_to_improve_attractiveness'],
      dtype='object')